In [1]:
suppressMessages(library(ggplot2))
suppressMessages(library(ArchR))
suppressMessages(library(Seurat))
suppressMessages(library(Signac))
suppressMessages(library(patchwork))
suppressMessages(library(dplyr))
suppressMessages(library(cowplot))
suppressMessages(library(ggrepel))
suppressMessages(library(rhdf5))

Warning message:
“package ‘devtools’ was built under R version 4.3.3”


In [2]:
set.seed(42)
addArchRThreads(threads = 64)

Setting default number of Parallel threads to 64.



In [3]:
proj <- loadArchRProject("./fragments/ArchRProject", showLogo = FALSE)
proj

Successfully loaded ArchRProject!


           ___      .______        ______  __    __  .______      
          /   \     |   _  \      /      ||  |  |  | |   _  \     
         /  ^  \    |  |_)  |    |  ,----'|  |__|  | |  |_)  |    
        /  /_\  \   |      /     |  |     |   __   | |      /     
       /  _____  \  |  |\  \\___ |  `----.|  |  |  | |  |\  \\___.
      /__/     \__\ | _| `._____| \______||__|  |__| | _| `._____|
    



class: ArchRProject 
outputDirectory: /home/xwt/27/BCY/BCY-pair/data_integration/fragments/ArchRProject 
samples(6): 10N 10T ... 12N 22T
sampleColData names(1): ArrowFiles
cellColData names(19): Sample TSSEnrichment ... Clusters
  predictedCellType
numberOfCells(1): 49601
medianTSS(1): 17.261
medianFrags(1): 15163

In [4]:
obj.atac.annotated <- readRDS("./fragments/ArchRProject/scATAC.annotated.Rds")

In [5]:
new_order <- c("T cells", "B cells", "Myeloid", "Mast", "Endothelial", "Plasmablast", "Epithelial", "Fibroblast", "PeriVascular")
obj.atac.annotated$majorType <- factor(obj.atac.annotated$majorType, levels = new_order)
cols <- ArchR::paletteDiscrete(obj.atac.annotated$majorType)
options(repr.plot.width = 7, repr.plot.height = 6)
p3 <- DimPlot(object = obj.atac.annotated, reduction = "umap_harmony",
              group.by = "majorType", shuffle = TRUE) +
              scale_color_manual(values = cols) +
              xlab("UMAP1") + ylab("UMAP2") + 
              ggtitle("") +
              theme(axis.title = element_blank(), axis.line = element_blank(), axis.ticks = element_blank(), axis.text = element_blank())

ggsave("./output/CellType_UMAP.svg", plot = p3, width = 7, height = 6, device = "svg")

In [6]:
df <- as.data.frame(obj.atac.annotated@meta.data)
length(unique(df$Sample))

df_ct <- df %>%
    group_by(Sample, majorType) %>%
    summarise(counts = n()) %>%
    mutate(cell_proportion = counts / sum(counts))

df_ct$majorType <- factor(df_ct$majorType, levels = new_order)
cols <- ArchR::paletteDiscrete(df_ct$majorType)

p2 <- ggplot(df_ct, aes(Sample, cell_proportion, fill=majorType)) + 
    geom_bar(stat="identity", position = position_stack(reverse = TRUE)) +
    scale_fill_manual(values = cols) +
    theme_cowplot() +
    xlab("") + ylab("") +
    theme(axis.text.x = element_text(angle = 60, hjust = 1)) +
    scale_y_continuous(labels = scales::percent_format(scale = 100), 
                       expand = expansion(mult = c(0, 0.05))) + 
    coord_cartesian(ylim = c(0, 1))
    

options(repr.plot.width = 6, repr.plot.height = 6)

ggsave("./output/CellType_proportion_sample.svg", plot = p2, width = 6, height = 6, device = "svg")

[1] 6

`summarise()` has grouped output by 'Sample'. You can override using the
`.groups` argument.


In [7]:
markersGS <- readRDS("./fragments/ArchRProject_clean/markersGS_predictedCellType.rds")

In [11]:
markerGenes <- c(
    "CD38", "IGLL5", "PAX5", "ITGAM", "FAP",
    "FOXP3", "VIM", "PDCD1", "CD8A", "CD3D",
    "CD4", "CXCL12", "THY1", "PECAM1", "KDR",
    "EPACM", "ERBB2", "FOXA1", "KRT8", "ESR1",
    "KRT14"
)

heatmapGS <- plotMarkerHeatmap(
  seMarker = markersGS, 
  cutOff = "FDR <= 0.05 & Log2FC >= 1.25", 
  labelMarkers = markerGenes,
  clusterCols = FALSE
)

idx <- which(mcols(markersGS)$name %in% markerGenes)
mtx <- assays(markersGS[idx,])$Mean %>% `rownames<-`(., mcols(markersGS)$name[idx])
mtx <- t(scale(t(mtx)))
fh <- function(x) hclust(dist(x), method="ward.D2")
col_fun1 <- colorRamp2(c(-2,-1,0,1,2), paletteContinuous(set = "horizonExtra", n = 5))


pdf("./output/heatmapGS_z_CellType.pdf", width = 6, height = 8)
options(repr.plot.height = 8, repr.plot.width = 6)
ht1 <- Heatmap(mtx, 
        name = "heatmapGS_z",
        cluster_rows = fh, 
        cluster_columns = F, 
        show_row_dend = F, 
        col = col_fun1,
        row_names_gp = gpar(fontsize = 12),
        column_names_gp = gpar(fontsize = 12),
        heatmap_legend_param = list(
                 title_gp = gpar(fontsize = 12,
                 labels_gp = gpar(fontsize = 10),
                 legend_direction = "horizontal", 
                 legend_width = unit(4, "cm"),
                 legend_height = unit(3, "cm"),
                 title_position = "topcenter"
               )
        )
      )
p3 <- draw(ht1, heatmap_legend_side = "bottom")

dev.off()

ArchR logging to : ArchRLogs/ArchR-plotMarkerHeatmap-1e41c7423248d5-Date-2025-11-09_Time-15-30-24.040861.log
If there is an issue, please report to github with logFile!



Printing Top Marker Genes:

B cells:

	MIR4420, RNU5F-1, BEND5, MIR4422, MIR7852, SLC16A4, DRAM2, TTC24, RALGPS2, MIR29C, FAM177B, KMO, OR14I1, IDI2-AS1, USP6NL

Endothelial:

	VWA1, LINC01770, RBP7, EPHA2, HSPG2, FAM110D, SLFNL1-AS1, TIE1, ADGRL4, LINC01361, ABCA4, LINC01708, PALMD, FAM72D, LCE3E

Epithelial:

	RNF223, MIR200B, MIR200A, MIR429, MIR6727, TAS1R3, ATAD3C, ARHGEF16, HES2, ESPN, MIR4252, CASZ1, PADI3, KLHDC7A, AKR7L

Fibroblast:

	GPR153, LINC01672, MIR34A, MIR6729, PRAMEF2, PRAMEF17, PRAMEF20, LRRC38, PDPN, PLEKHM2, PLA2G5, FAM43B, MIR4684, TCEA3, A3GALT2

Mast:

	UTS2, NPPA-AS1, PLA2G2C, TRIM63, MIR6735, NFIA-AS2, MIR3117, TCTEX1D1, ALX3, ATP1A4, RGS13, LHX9, SLC45A3, SPATA45, GCSAML

Myeloid:

	TNFRSF14, MIR4632, PADI2, CELA3A, MIR6127, C1QC, C1QB, FGR, PTAFR, FAM151A, DNASE2B, GNAT2, AMPD2, CTSS, RPTN

PeriVascular:

	LINC00982, PRDM16, NPPB, FBLIM1, UQCRHL, GJA4, NT5C1A, CYP4X1, SGIP1, CYR61, PLPPR4, INKA2-AS1, ETV3L, OR10R2, OR6Y1

Plasmablast:

	SLC2A7, CORT, PLA2G2

 [1] "CD38"   "IGLL5"  "PAX5"   "ITGAM"  "FAP"    "FOXP3"  "VIM"    "PDCD1" 
 [9] "CD8A"   "CD3D"   "CD4"    "CXCL12" "THY1"   "PECAM1" "KDR"    "EPACM" 
[17] "ERBB2"  "FOXA1"  "KRT8"   "ESR1"   "KRT14" 


Preparing Main Heatmap..

'magick' package is suggested to install to give better rasterization.

Set `ht_opt$message = FALSE` to turn off this message.

ArchR logging successful to : ArchRLogs/ArchR-plotMarkerHeatmap-1e41c7423248d5-Date-2025-11-09_Time-15-30-24.040861.log



agg_record_180837382 
                   2

In [12]:
## motif analysis - Cell Type
VarMotifs <- getVarDeviations(proj, name = "homerMatrix", plot = F)
MotifScoreClusters <- getMarkerFeatures(proj, 
                                        useMatrix = "homerMatrix", 
                                        groupBy = "predictedCellType",
                                        bias = c("TSSEnrichment", "log10(nFrags)"),
                                        testMethod = "wilcoxon", 
                                        useSeqnames = "z")
fh <- function(x) hclust(dist(x), method="ward.D2")
idy <- which(mcols(MotifScoreClusters)$name %in% VarMotifs$name[c(1:50)])
mtx2 <- assays(MotifScoreClusters[idy,])$Mean %>% `rownames<-`(., mcols(MotifScoreClusters)$name[idy])
col_fun2 <- colorRamp2(c(-5,-2.5,0,2,5), paletteContinuous(set = "solarExtra", n = 5))
pdf("./output/heatmap_Motif_CellType.pdf", width = 8, height = 8)
ht2 <- Heatmap(mtx2, 
        name = "Mean motif score", 
        cluster_rows = fh, 
        cluster_columns = F, 
        show_row_dend = F, 
        col = col_fun2,
        row_names_gp = gpar(fontsize = 12),
        column_names_gp = gpar(fontsize = 12),
        heatmap_legend_param = list(
                 title_gp = gpar(fontsize = 12),
                 labels_gp = gpar(fontsize = 10), 
                 legend_direction = "horizontal",
                 legend_width = unit(4, "cm"),
                 legend_height = unit(3, "cm"),
                 title_position = "topcenter"
               )
        )

p6 <- draw(ht2, heatmap_legend_side = "bottom")
p6
dev.off()

DataFrame with 6 rows and 6 columns
    seqnames       idx                   name combinedVars combinedMeans
       <Rle> <integer>            <character>    <numeric>     <numeric>
f90        z        90 Fox.Ebox.Forkhead.bH..      67.6093     -0.315460
f87        z        87      FOXA1.Forkhead_87      64.6130     -0.331883
f1         z         1            AP.1.bZIP_1      60.7542     -0.300232
f86        z        86          Fosl2.bZIP_86      59.4828     -0.309931
f94        z        94      FOXM1.Forkhead_94      59.3852     -0.299776
f99        z        99           Fra2.bZIP_99      58.8002     -0.330543
         rank
    <integer>
f90         1
f87         2
f1          3
f86         4
f94         5
f99         6


ArchR logging to : ArchRLogs/ArchR-getMarkerFeatures-1e41c760c972e0-Date-2025-11-09_Time-15-31-01.413643.log
If there is an issue, please report to github with logFile!

MatrixClass = Sparse.Assays.Matrix

2025-11-09 15:31:01.585875 : Matching Known Biases, 0.002 mins elapsed.

Found less than 100 cells for background matching, Lowering k to 0

2025-11-09 15:31:04.470547 : 



In [13]:
## motif analysis - cell types motif enrichment
PeakCellType <- readRDS("./fragments/ArchRProject_clean/markersPK_predictedCellType.rds")
PeakCellType

class: SummarizedExperiment 
dim: 298721 9 
metadata(2): MatchInfo Params
assays(7): Log2FC Mean ... AUC MeanBGD
rownames(298721): 1 2 ... 298720 298721
rowData names(4): seqnames idx start end
colnames(9): B cells Endothelial ... Plasmablast T cells
colData names(0):

In [14]:
heatmapPeaks <- plotMarkerHeatmap(
    seMarker = PeakCellType,
    cutOff = "FDR < 0.01 & Log2FC >= 1",
    transpose = TRUE,
    clusterCols = FALSE,
    nLabel = 0
)
options(repr.plot.height = 10, repr.plot.width = 10)

pdf("./output/heatmapPK_CellType.pdf", width = 10, height = 10)
ComplexHeatmap::draw(heatmapPeaks, 
                     heatmap_legend_side = "bot", 
                     annotation_legend_side = "bot")
dev.off()

ArchR logging to : ArchRLogs/ArchR-plotMarkerHeatmap-1e41c72cbeb47-Date-2025-11-09_Time-15-31-17.738615.log
If there is an issue, please report to github with logFile!



Identified 153244 markers!



character(0)


Adding Annotations..

Preparing Main Heatmap..

'magick' package is suggested to install to give better rasterization.

Set `ht_opt$message = FALSE` to turn off this message.

ArchR logging successful to : ArchRLogs/ArchR-plotMarkerHeatmap-1e41c72cbeb47-Date-2025-11-09_Time-15-31-17.738615.log



agg_record_1120477757 
                    2

In [15]:
enrichMotifs <- ArchR::peakAnnoEnrichment(
  seMarker = PeakCellType,
  ArchRProj = proj,
  peakAnnotation = "homer",
  cutOff = "FDR < 0.01 & Log2FC >= 1"
)
enrichMotifs
heatmapMotif <- plotEnrichHeatmap(enrichMotifs, n = 7, clusterCols = F, transpose = TRUE)
options(repr.plot.height = 10, repr.plot.width = 10)
pdf("./output/heatmapPK_motif_CellType.pdf", width = 10, height = 10)
ComplexHeatmap::draw(heatmapMotif, 
                     heatmap_legend_side = "bot", 
                     annotation_legend_side = "bot")
dev.off()

ArchR logging to : ArchRLogs/ArchR-peakAnnoEnrichment-1e41c71b68af92-Date-2025-11-09_Time-15-31-34.217354.log
If there is an issue, please report to github with logFile!



2025-11-09 15:31:45.598196 : Computing Enrichments 1 of 9, 0.19 mins elapsed.

2025-11-09 15:31:45.800749 : Computing Enrichments 2 of 9, 0.193 mins elapsed.

2025-11-09 15:31:45.956959 : Computing Enrichments 3 of 9, 0.196 mins elapsed.

2025-11-09 15:31:46.134739 : Computing Enrichments 4 of 9, 0.199 mins elapsed.

2025-11-09 15:31:46.30435 : Computing Enrichments 5 of 9, 0.201 mins elapsed.

2025-11-09 15:31:46.448942 : Computing Enrichments 6 of 9, 0.204 mins elapsed.

2025-11-09 15:31:46.608091 : Computing Enrichments 7 of 9, 0.207 mins elapsed.

2025-11-09 15:31:46.76103 : Computing Enrichments 8 of 9, 0.209 mins elapsed.

2025-11-09 15:31:46.924539 : Computing Enrichments 9 of 9, 0.212 mins elapsed.

ArchR logging successful to : ArchRLogs/ArchR-peakAnnoEnrichment-1e41c71b68af92-Date-2025-11-09_Time-15-31-34.217354.log



class: SummarizedExperiment 
dim: 332 9 
metadata(0):
assays(10): mlog10Padj mlog10p ... CompareFrequency feature
rownames(332): AP.1.bZIP_1 AP.2gamma.AP2_2 ... ZNF711.Zf_331
  ZSCAN22.Zf_332
rowData names(0):
colnames(9): B cells Endothelial ... Plasmablast T cells
colData names(0):

ArchR logging to : ArchRLogs/ArchR-plotEnrichHeatmap-1e41c774af163e-Date-2025-11-09_Time-15-31-47.204026.log
If there is an issue, please report to github with logFile!

Adding Annotations..

Preparing Main Heatmap..

'magick' package is suggested to install to give better rasterization.

Set `ht_opt$message = FALSE` to turn off this message.



agg_record_1726491565 
                    2

In [16]:
markerGenes <- c(
    "ADARB1", 
    "POFUT2", 
    "SKA2", 
    "YPEL2",
    "CDH1"
)

p <- plotBrowserTrack(
    ArchRProj = proj, 
    groupBy = "Sample", 
    geneSymbol = markerGenes, 
    upstream = 50000,
    downstream = 50000
)

ArchR logging to : ArchRLogs/ArchR-plotBrowserTrack-1e41c734fe364e-Date-2025-11-09_Time-15-31-49.924781.log
If there is an issue, please report to github with logFile!

2025-11-09 15:31:50.001576 : Validating Region, 0.001 mins elapsed.



GRanges object with 5 ranges and 2 metadata columns:
      seqnames            ranges strand |     gene_id      symbol
         <Rle>         <IRanges>  <Rle> | <character> <character>
  [1]    chr21 45073853-45226560      + |         104      ADARB1
  [2]    chr21 45263928-45287898      - |       23275      POFUT2
  [3]    chr17 59109951-59155269      - |      348235        SKA2
  [4]    chr17 59331689-59401729      + |      388403       YPEL2
  [5]    chr16 68737225-68835548      + |         999        CDH1
  -------
  seqinfo: 24 sequences from hg38 genome


2025-11-09 15:31:50.07093 : Adding Bulk Tracks (1 of 5), 0.002 mins elapsed.



R_zmq_msg_send errno: 4 strerror: Interrupted system call


2025-11-09 15:31:57.05232 : Adding Feature Tracks (1 of 5), 0.119 mins elapsed.

2025-11-09 15:31:57.157979 : Adding Gene Tracks (1 of 5), 0.121 mins elapsed.

2025-11-09 15:31:57.499059 : Plotting, 0.126 mins elapsed.

2025-11-09 15:31:58.614536 : Adding Bulk Tracks (2 of 5), 0.145 mins elapsed.

2025-11-09 15:32:00.04298 : Adding Feature Tracks (2 of 5), 0.169 mins elapsed.

2025-11-09 15:32:00.11223 : Adding Gene Tracks (2 of 5), 0.17 mins elapsed.

2025-11-09 15:32:00.479774 : Plotting, 0.176 mins elapsed.

2025-11-09 15:32:01.389142 : Adding Bulk Tracks (3 of 5), 0.191 mins elapsed.

2025-11-09 15:32:04.986119 : Adding Feature Tracks (3 of 5), 0.251 mins elapsed.

2025-11-09 15:32:05.069726 : Adding Gene Tracks (3 of 5), 0.252 mins elapsed.

2025-11-09 15:32:05.399436 : Plotting, 0.258 mins elapsed.

2025-11-09 15:32:06.277761 : Adding Bulk Tracks (4 of 5), 0.273 mins elapsed.

2025-11-09 15:32:08.483752 : Adding Feature Tracks (4 of 5), 0.309 mins elapsed.

2025-11-09 15:32:08.55

In [17]:
plotPDF(plotList = p, 
    name = "Plot-Tracks-Marker-Genes.pdf", 
    ArchRProj = proj, 
    addDOC = FALSE, width = 5, height = 5)

Plotting Gtable!



NULL


Plotting Gtable!



NULL


Plotting Gtable!



NULL


Plotting Gtable!



NULL


Plotting Gtable!



NULL


In [18]:
sessionInfo()

R version 4.3.1 (2023-06-16)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 22.04.2 LTS

Matrix products: default
BLAS/LAPACK: /home/xwt/miniconda3/envs/YBC_NAT/lib/libopenblasp-r0.3.30.so;  LAPACK version 3.12.0

Random number generation:
 RNG:     L'Ecuyer-CMRG 
 Normal:  Inversion 
 Sample:  Rejection 
 
locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Etc/UTC
tzcode source: system (glibc)

attached base packages:
 [1] parallel  stats4    grid      stats     graphics  grDevices utils    
 [8] datasets  methods   base     

other attached packages:
 [1] nabor_0.5.0                 circlize_0.4.16            
 [3] ComplexHeatmap_2.10.0       ggrepel